In [1]:


import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01  # 学习率
EPSILON = 0.95  # 贪心策略
GAMMA = 0.8  # 奖励衰减
TARGET_REPLACE_ITER = 100  # 目标更新频率
MEMORY_CAPACITY = 2000
env = gym.make('MountainCar-v0')
# env = gym.make('CartPole-v1')
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
# ENV_A_SHAPE = 0 if isinstance(env.action_space.sample()[0], int) else env.action_space.sample().shape  # 确认形状
print(N_STATES)
print(N_ACTIONS)

2
3


In [3]:
a = torch.randn(4, 6)
a

tensor([[-0.8934,  1.4577, -0.3254, -0.2330, -0.3444, -1.5549],
        [ 1.6870,  0.2964, -0.8691,  0.8560,  0.8739, -1.2244],
        [-1.5050,  1.0970,  0.6678,  0.3284,  0.2981,  0.6359],
        [ 0.8935,  1.5149,  1.0219,  1.4659, -0.3375,  1.0385]])

In [4]:
torch.max(a, 1)[1]

tensor([1, 0, 1, 1])

In [5]:
class DQN(object):  # 强化神经网络
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()  # 定义两个网络：评估网络 & 目标网络
        self.learn_step_counter = 0  # 用于目标网络延迟更新
        self.memory_counter = 0  # 存储计数器
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))  # 初始化记忆全为 0
        # 记忆单元为 当前状态 行为 奖励 操作后状态
        # 单元数为 状态维度 * 2 + 行为维度 + 奖励维度(1)
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)  # todo Adam 优化器
        self.loss_func = nn.MSELoss()  # 均方误差损失函数

    def choose_action(self, x):  # 传入当前的 State，计算行为
        x = torch.unsqueeze(torch.FloatTensor(x), 0)  # 解包
        # 只输入一个样本
        if np.random.uniform() < EPSILON:  # 90% 概率使用评估网络的结果
            actions_value = self.eval_net.forward(x)  # 使用评估网络获取行为
            # actions_value = [a1,b1,c1,d1]
            # actions_value = [a1,b1,c1,d1]
            action = torch.max(actions_value, 1)[1].data.numpy()  # 每行的最大值索引
            action = action[0]  #if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
            # 返回argmax索引
        else:  # 随机
            action = np.random.randint(0, N_ACTIONS)
            action = action  #if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        # print('[state]:',s)
        # print('[action,reward]:',[a, r])
        # print('[stated]:',s_)
        transition = np.hstack((s, [a, r], s_))
        # 用新内存替换旧内存
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # 目标参数更新
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # 样本批次转换
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES + 1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES + 1:N_STATES + 2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t体验中的行动
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()  # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)  # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [6]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 1)  # 传入网络 状态数 -> 50
        self.fc1.weight.data.normal_(0, 0.1)  # 随机初始化权重
        self.out = nn.Linear(1, N_ACTIONS)  # 输出网络 50 -> 输出Action数
        self.out.weight.data.normal_(0, 0.1)  # 随机初始化权重

    def forward(self, x):  # 前向传播
        x = self.fc1(x)  # 第一层映射
        x = F.relu(x)  # 激活函数 线性整流函数 f(x) = max(0,x)
        actions_value = self.out(x)  # 输出映射
        return actions_value

dqn = DQN()

In [ ]:
print('\nCollecting experience...')
import time

# try:
#     print("读取模型中...")
#     dqn.target_net = torch.load("target.pth")
#     dqn.eval_net = torch.load("eval.pth")
#     print("读取模型成功!")
# except Exception as _:
#     print("读取模型失败!")
flag = True
for i_episode in range(400):
    if flag:
        s = env.reset()
        ep_r = 0
        times = 0
        start = time.time()
        max_x = -1.2
        min_x = 0.6
        while True:
            try:
                env.render()
                a = dqn.choose_action(s)
                # if x >= 0.0 and v <= 0.0:  # 右边下滑
                #     a = 0  # 向左加速
                # elif x >= 0.0 and v > 0.0:  # 右边上冲
                #     a = 2  # 向右加速
                # elif x < 0.0 and v >= 0.0:  # 左边下滑
                #     a = 2  # 向右加速
                # elif x < 0.0 and v < 0.0:  # 左边上冲
                #     a = 0  # 向左加速
                # take action
                s_, r, done, info = env.step(a)
                x, v = s_
                # if x > -0.5:
                #     r = abs(x * 5)
                # else:
                #     r = abs(x * 5)
                # r = (r * 1000) / times
                # x = x + 0.5
                if x >= 0.0 and v <= 0.0:  # 右边下滑
                    if a == 0:  # 向左加速
                        r = x
                    else:
                        r = -x
                elif x >= 0.0 and v > 0.0:  # 右边上冲
                    if a == 2:  # 向右加速
                        r = x
                    else:
                        r = -x
                elif x < 0.0 and v >= 0.0:  # 左边下滑
                    if a == 2:  # 向右加速
                        r = -x
                    else:
                        r = x
                elif x < 0.0 and v < 0.0:  # 左边上冲
                    if a == 0:  # 向左加速
                        r = -x
                    else:
                        r = x
                # if a == 1:
                #     r = 0
                # r = x
                if x > max_x: max_x = x
                if x < min_x: min_x = x
                if times % 100 == 0:
                    print(times)
                #     print('[', times, ']{(x,r):(', round(x, 2), ',', round(r, 2), ')},{range:[', round(min_x, 2), ',',
                #           round(max_x, 2), ']}')
                times += 1
                # print(str(times) + ":" + str(r))
                # r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
                # r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
                # r = r1 + r2

                dqn.store_transition(s, a, r, s_)

                ep_r += r
                if dqn.memory_counter > MEMORY_CAPACITY:
                    dqn.learn()
                    if done:
                        t = time.time() - start
                        print('轮数: ', i_episode,
                              '| 次数: ', round(times, 2),
                              '| 奖励: ', round(ep_r, 2),
                              '| 结果: ', round(x, 4),
                              '| 速度: ', round(v, 4),
                              '| 时间: ', round(t, 2), 's'
                              )

                if done:
                    break
                s = s_
            except KeyboardInterrupt as e:
                flag = False
                break
    else:
        break

print("保存模型中...")
torch.save(dqn.target_net, "target.pth")
torch.save(dqn.eval_net, "eval.pth")
print("保存成功，程序已退出")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
轮数:  0 | 次数:  2371 | 奖励:  22.71 | 结果:  0.5369 | 速度:  0.05 | 时间:  40.19 s
0
100
轮数:  1 | 次数:  178 | 奖励:  67.75 | 结果:  0.5283 | 速度:  0.0488 | 时间:  2.97 s
0
100
轮数:  2 | 次数:  148 | 奖励:  66.61 | 结果:  0.5054 | 速度:  0.0492 | 时间:  2.47 s
0
100
轮数:  3 | 次数:  111 | 奖励:  55.28 | 结果:  0.5009 | 速度:  0.0482 | 时间:  1.85 s
0
100
轮数:  4 | 次数:  118 | 奖励:  56.51 | 结果:  0.5391 | 速度:  0.0402 | 时间:  1.97 s
0
轮数:  5 | 次数:  86 | 奖励:  44.07 | 结果:  0.5228 | 速度:  0.0258 | 时间:  1.43 s
0
100
轮数:  6 | 次数:  116 | 奖励:  64.73 | 结果:  0.5239 | 速度:  0.0456 | 时间:  1.93 s
0
100
轮数:  7 | 次数:  152 | 奖励:  74.24 | 结果:  0.5162 | 速度:  0.0485 | 时间:  2.53 s
0
100
轮数:  8 | 次数:  157 | 奖励:  73.78 | 结果:  0.5369 | 速度:  0.05 | 时间:  2.62 s
0
100
轮数:  9 | 次数:  151 | 奖励:  69.7 | 结果:  0.5162 | 速度:  0.0485 | 时间:  2.52 s
0
轮数:  10 | 次数:  88 | 奖励:  43.36 | 结果:  0.5165 | 速度:  0.0245 | 时间:  1.47 s
0
100
轮数:  11 | 次数:  161 | 奖励:  74.97 | 

In [ ]:
def test():
    s = env.reset()
    ep_r = 0
    times = 0
    start = time.time()
    max_x = -1.2
    min_x = 0.6
    while True:
        try:
            # env.render()
            a = dqn.choose_action(s)
            s_, r, done, info = env.step(a)
            x, v = s_
            # r = x
            if x > max_x: max_x = x
            if x < min_x: min_x = x
            # if times % 10 == 0:
            #     print('[', times, ']{(x,v):(', round(x, 2), ',', round(v, 2), ')},{range:[', round(min_x, 2), ',',
            #           round(max_x, 2), ']}')
            times += 1
            ep_r += r
            if done:
                t = time.time() - start
                print('次数: ', round(times, 2),
                      '| 结果: ', round(x, 4),
                      '| 速度: ', round(v, 4),
                      '| 时间: ', round(t, 2), 's'
                      )
                return times
            s = s_
        except KeyboardInterrupt as e:
            break

In [ ]:
try:
    print("读取模型中...")
    dqn.target_net = torch.load("target.pth")
    dqn.eval_net = torch.load("eval.pth")
    print("读取模型成功!")
except Exception as _:
    print("读取模型失败!")
t = 0
for _ in range(0,20):
    t += test()
print("累计次数: ",t)

In [ ]:
try:
    print("读取模型中...")
    dqn.target_net = torch.load("../target.pth")
    dqn.eval_net = torch.load("../eval.pth")
    print("读取模型成功!")
except Exception as _:
    print("读取模型失败!")
t = 0
for _ in range(0,20):
    t += test()
print("累计次数: ",t)